In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install fastai==1.0.61
!pip install torchsummary

     |████████████████████████████████| 239 kB 802 kB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19191 sha256=db0f87e27b1a8553ae2b750457aac32499dd7f57c7b003608b409e6922667bdd
  Stored in directory: /root/.cache/pip/wheels/df/99/da/c34f202dc8fd1dffd35e0ecf1a7d7f8374ca05fbcbaf974b83
Successfully built nvidia-ml-py3
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: fastai
    Found existing installation: fastai 2.5.3
    Uninstalling fastai-2.5.3:
      Successfully uninstalled fastai-2.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
# from google.colab import drive
# drive.mount('/content/drive')
from glob import glob
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary
from fastai.metrics import accuracy
from fastai.vision import *
import numpy as np
!pip install fft-conv-pytorch
from fft_conv_pytorch import fft_conv, FFTConv2d

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from fastai.vision.data import normalize
path = "../input/covid19grayscaleds/covid-19_dataset_grayscale"
np.random.seed(41)
defaults = SimpleNamespace(cmap='L', return_fig=False, silent=False)
data = ImageDataBunch.from_folder(path, train="Train", valid ="Valid", valid_pct=0.2,ds_tfms=get_transforms(), size=(256,256), bs=16, num_workers=4)
for itemList in ["train_dl", "valid_dl", "fix_dl", "test_dl"]:
    itemList = getattr(data, itemList)
    if itemList: itemList.x.convert_mode = "L"

In [ ]:
print (data.one_batch()[0].shape)
print (data.classes, data.c, len(data.train_ds), len(data.valid_ds))
print (data.train_ds.classes)
print (data.valid_ds.classes)

torch.Size([16, 1, 256, 256])
['covid-19', 'no_findings', 'pneumonia'] 3 900 225
['covid-19', 'no_findings', 'pneumonia']
['covid-19', 'no_findings', 'pneumonia']


With FFT Conv Layers

In [ ]:
def fft_conv_block(ni, nf, size=3, stride=1):
    for_pad = lambda s: s if s > 2 else 3
    return nn.Sequential(
        FFTConv2d(ni, nf, kernel_size=size, stride=stride,
                  padding=(for_pad(size) - 1)//2, bias=False), 
        nn.BatchNorm2d(nf),
        nn.LeakyReLU(negative_slope=0.1, inplace=True)  
    )

def conv_block(ni, nf, size=3, stride=1):
    for_pad = lambda s: s if s > 2 else 3
    return nn.Sequential(
        nn.Conv2d(ni, nf, kernel_size=size, stride=stride,
                  padding=(for_pad(size) - 1)//2, bias=False), 
        nn.BatchNorm2d(nf),
        nn.LeakyReLU(negative_slope=0.1, inplace=True)  
    )

def triple_conv(ni, nf):
    return nn.Sequential(
        conv_block(ni, nf),
        conv_block(nf, ni, size=1),  
        conv_block(ni, nf)
    )
def fft_triple_conv(ni, nf,size):
    return nn.Sequential(
        fft_conv_block(ni, nf,size),
        conv_block(nf, ni, size=1),  
        fft_conv_block(ni, nf, size)
    )
def maxpooling():
    return nn.MaxPool2d(2, stride=2)

In [ ]:
fft_model4 = nn.Sequential(
    fft_conv_block(1, 8, 25),
    maxpooling(),
    conv_block(8, 16),
    maxpooling(),
    triple_conv(16,32),
    maxpooling(),
    triple_conv(32, 64),
    maxpooling(),
    triple_conv(64, 128),
    maxpooling(),
    triple_conv(128, 256),
    conv_block(256, 128, size=1),
    conv_block(128, 256),
    conv_block(256,3),
    nn.Flatten(),
    nn.Linear(1323, 3)
)
fft_learn4 = Learner(data, fft_model4, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)
print(fft_learn4.summary())
fft_learn4.fit_one_cycle(100, max_lr=4e-3)
probs,targets = fft_learn4.get_preds(ds_type=DatasetType.Valid) 
accuracy(probs,targets)

Sequential
Layer (type)         Output Shape         Param #    Trainable 
_FFTConv             [8, 512, 512]        5,000      True      
______________________________________________________________________
BatchNorm2d          [8, 512, 512]        16         True      
______________________________________________________________________
LeakyReLU            [8, 512, 512]        0          False     
______________________________________________________________________
MaxPool2d            [8, 256, 256]        0          False     
______________________________________________________________________
Conv2d               [16, 256, 256]       1,152      True      
______________________________________________________________________
BatchNorm2d          [16, 256, 256]       32         True      
______________________________________________________________________
LeakyReLU            [16, 256, 256]       0          False     
___________________________________________________

epoch,train_loss,valid_loss,accuracy,time
0,0.856709,0.668765,0.728889,00:33
1,0.735656,0.495665,0.831111,00:33
2,0.674400,0.525266,0.786667,00:33
3,0.666389,0.530127,0.737778,00:33
4,0.642623,0.745960,0.635556,00:32
5,0.640397,0.595539,0.724444,00:33
6,0.649613,0.628429,0.773333,00:33
7,0.669420,1.635563,0.488889,00:33
8,0.683828,0.610833,0.755556,00:33
9,0.632662,0.798651,0.595556,00:33


tensor(0.8978)